<a href="https://colab.research.google.com/github/DeveloperPratim/sses_final/blob/main/LastCommitSSES_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch sentence-transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import math
import re
from sentence_transformers import SentenceTransformer, util  # For semantic similarityimport math

# Global variables for models and tokenizers
model_qwen = None
tokenizer_qwen = None
sentence_transformer = None
models_loaded = False  # Flag to ensure models are loaded only once


def load_models():
    """
    Load global models and tokenizers.
    This function should be called only once.
    """
    global model_qwen, tokenizer_qwen, sentence_transformer, models_loaded

    if not models_loaded:  # Load models only if not already loaded
        print("Loading models...")

        # Load Qwen model
        drive_path_qwen = "/content/drive/MyDrive/sses/Qwen2.5-1.5B-Instruct"
        tokenizer_qwen = AutoTokenizer.from_pretrained(drive_path_qwen, local_files_only=True)
        model_qwen = AutoModelForCausalLM.from_pretrained(drive_path_qwen, local_files_only=True).to(torch.device("cpu"))

        # Load SentenceTransformer for semantic similarity (MiniLM model)
        drive_path_minilm = "/content/drive/MyDrive/sses/all-MiniLM-L6-v2"
        sentence_transformer = SentenceTransformer(drive_path_minilm, device="cpu")

        models_loaded = True
        print("Models loaded successfully.")
    else:
        print("Models are already loaded.")


def generate_response(question, student_answer, marks):
    """
    Generate a response using the Qwen model.

    :param question: The input question.
    :param student_answer: The input student's answer.
    :return: The generated text response.
    """
    if not models_loaded:
        raise RuntimeError("Models are not loaded. Call `load_models()` first.")

    # Prepare the input text
    input_text = f"""
    Question: {question}
    Answer: {student_answer}
    Evaluate and provide these two fields **score** and **feedback** based on this answer where full marks is {marks} and can be given for the most precise and best answer, and 0 for wrong or irrelevant answers. Respond only with these two fields in JSON format.
    """

    # Tokenize the input
    input_ids = tokenizer_qwen.encode(input_text, return_tensors="pt").to(torch.device("cpu"))

    # Dynamically set the maximum number of new tokens
    max_allowed_tokens = model_qwen.config.max_position_embeddings  # Total token limit of the model
    input_token_count = len(input_ids[0])
    max_new_tokens = max(50, max_allowed_tokens - input_token_count)  # Minimum 50 new tokens

    # Generate the response
    output_ids = model_qwen.generate(
        input_ids,
        max_length=input_token_count + max_new_tokens,  # Total length: input + generated tokens
        num_return_sequences=1,
        no_repeat_ngram_size=2,  # Avoid repetition
        early_stopping=True,    # Stop when the output is complete
    )

    # Decode and return the generated output
    return tokenizer_qwen.decode(output_ids[0], skip_special_tokens=True)


def extract_score_and_feedback(generated_text):
    """
    Extract score and feedback from the generated text using regular expressions.

    :param generated_text: The raw output from the model.
    :return: A dictionary with "score" and "feedback".
    """
    score_pattern = r'"score":\s*([0-9.]+)'  # Matches numeric values for score
    feedback_pattern = r'"feedback":\s*"([^"]+)"'  # Matches the feedback string

    # Find matches
    score_match = re.search(score_pattern, generated_text)
    feedback_match = re.search(feedback_pattern, generated_text)

    # Extract values
    score = float(score_match.group(1)) if score_match else None  # Convert to float if match found
    feedback = feedback_match.group(1) if feedback_match else None  # Extract feedback text if match found

    return {"score": score, "feedback": feedback}


def calculate_semantic_similarity(student_answer, expected_answer):
    """
    Calculate the semantic similarity between the student's answer and the expected answer.

    :param student_answer: The student's answer.
    :param expected_answer: The expected correct answer.
    :return: The semantic similarity score.
    """
    if not models_loaded:
        raise RuntimeError("Models are not loaded. Call `load_models()` first.")

    # Encode both answers using the SentenceTransformer model
    embeddings = sentence_transformer.encode([student_answer, expected_answer], convert_to_tensor=True)

    # Compute cosine similarity
    similarity_score = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
    return similarity_score

"""

def GetData(question, student_answer, expected_answer, marks):
    '''
    Function to generate evaluation data based on the student's answer.

    :param question: The question asked in the exam.
    :param student_answer: The student's answer to the question.
    :param expected_answer: The expected correct answer for the question.
    :param marks: The full marks assigned to the question.

    :return: A dictionary containing the analysis result.
    '''
    if not models_loaded:
        raise RuntimeError("Models are not loaded. Call `load_models()` first.")

    # Generate response using Qwen model
    generated_text = generate_response(question, student_answer, marks)
    # Extract score and feedback
    result = extract_score_and_feedback(generated_text)
    score = result['score']
    feedback = result['feedback']

    # Calculate semantic similarity between student's answer and expected answer
    similarity = calculate_semantic_similarity(student_answer, expected_answer)

    # Calculate final marks obtained based on score and semantic similarity
    similarity_weight = 0.5  # You can adjust this weight based on your preference
    final_marks_obtained = (score * 0.5) + (similarity * similarity_weight)

    # Ensure that final_marks_obtained does not exceed the total marks
    final_marks_obtained = min(final_marks_obtained, marks)

    # Return the result as a dictionary
    result_data = {
        "question": question,
        "student_answer": student_answer,
        "expected_answer": expected_answer,
        "marks": marks,
        "score": score,
        "feedback": feedback,
        "similarity": similarity,
        "final_marks_obtained": final_marks_obtained
    }

    return result_data



    # Weighted sum of score and similarity
    weighted_score = log_score * 0.5  # Adjust weight as needed
    weighted_similarity = log_similarity * 0.5  # Adjust weight as needed

    # Calculate an initial final marks based on the weighted sum
    final_marks_obtained = weighted_score + weighted_similarity
    final_marks_obtained = final_marks_obtained * marks  # Scale by full marks

    # Apply penalty: if the final marks exceed 90% of full marks, reduce them
    max_allowed_marks = marks * 0.9  # No student should get more than 90% of full marks
    if final_marks_obtained > max_allowed_marks:
        penalty_factor = 0.8  # Apply a 20% penalty
        final_marks_obtained = final_marks_obtained * penalty_factor

    # Ensure that the final marks are never below 0
    final_marks_obtained = max(final_marks_obtained, 0)

    final_marks_obtained = ( final_marks_obtained  + score) /2


    # Calculate percentages
    final_marks_percentage = round((final_marks_obtained / marks) * 100, 2) if marks > 0 else 0
    score_percentage = round((score / marks) * 100, 2) if marks > 0 else 0

    # Analyze the student's answer
    student_word_count = len(student_answer.split())  # Count words in the student's answer
    student_sentence_count = len(re.split(r'[.!?]+', student_answer.strip())) - 1  # Count sentences

    # Return the result as a dictionary
    result_data = {
        "question": question,
        "student_answer": student_answer,
        "expected_answer": expected_answer,
        "marks": marks,
        "score": score,
        "score_percentage": score_percentage,
        "feedback": feedback,
        "similarity": similarity,
        "similarity_percentage": similarity_percentage,
        "final_marks_obtained": final_marks_obtained,
        "final_marks_percentage": final_marks_percentage,
        "student_word_count": student_word_count,
        "student_sentence_count": student_sentence_count
    }


"""


def GetData(question, student_answer, expected_answer, marks):
    """
    Function to generate evaluation data based on the student's answer.

    :param question: The question asked in the exam.
    :param student_answer: The student's answer to the question.
    :param expected_answer: The expected correct answer for the question.
    :param marks: The full marks assigned to the question.

    :return: A dictionary containing the analysis result.
    """
    if not models_loaded:
        raise RuntimeError("Models are not loaded. Call `load_models()` first.")

    # Generate response using Qwen model
    generated_text = generate_response(question, student_answer, marks)
    # Extract score and feedback
    result = extract_score_and_feedback(generated_text)
    score = result['score']
    feedback = result['feedback']

    # Calculate semantic similarity between student's answer and expected answer
    similarity = calculate_semantic_similarity(student_answer, expected_answer)
    similarity_percentage = round(similarity * 100, 2)  # Convert similarity to percentage

    # Logarithmic scaling of score and similarity
    log_score = math.log(score + 1)  # Logarithm of score (score + 1 to avoid log(0))
    log_similarity = math.log(similarity + 1)  # Logarithm of similarity (similarity + 1 to avoid log(0))

    # Weighted sum of score and similarity
    weighted_score = log_score * 0.5  # Adjust weight as needed
    weighted_similarity = log_similarity * 0.5  # Adjust weight as needed

    # Calculate an initial final marks based on the weighted sum
    final_marks_obtained = weighted_score + weighted_similarity
    final_marks_obtained = final_marks_obtained * marks  # Scale by full marks

    # Apply penalty: if the final marks exceed 90% of full marks, reduce them
    max_allowed_marks = marks * 0.9  # No student should get more than 90% of full marks
    if final_marks_obtained > max_allowed_marks:
        penalty_factor = 0.8  # Apply a 20% penalty
        final_marks_obtained = final_marks_obtained * penalty_factor

    # Ensure that the final marks are never below 0
    final_marks_obtained = max(final_marks_obtained, 0)

    final_marks_obtained = (final_marks_obtained + score) / 2

    # Calculate percentages
    final_marks_percentage = round((final_marks_obtained / marks) * 100, 2) if marks > 0 else 0
    score_percentage = round((score / marks) * 100, 2) if marks > 0 else 0

    # Analyze the student's answer
    student_word_count = len(student_answer.split())  # Count words in the student's answer
    student_sentence_count = len(re.split(r'[.!?]+', student_answer.strip())) - 1  # Count sentences

    # Check for plagiarism
    results_plagiarism = plagiarism_checker(student_answer)

    # Detect AI-generated text
    result_AI = detect_ai_generated_text_advanced_v3(student_answer, threshold=0.7)

    # Return the result as a dictionary, including plagiarism and AI-detection data
    result_data = {
        "question": question,
        "student_answer": student_answer,
        "expected_answer": expected_answer,
        "marks": marks,
        "score": score,
        "score_percentage": score_percentage,
        "feedback": feedback,
        "similarity": similarity,
        "similarity_percentage": similarity_percentage,
        "final_marks_obtained": final_marks_obtained,
        "final_marks_percentage": final_marks_percentage,
        "student_word_count": student_word_count,
        "student_sentence_count": student_sentence_count,
        "plagiarism": results_plagiarism["plagiarism"],
        "average_score": results_plagiarism["average_score"],
        "max_score": results_plagiarism["max_score"],
        "max_score_url": results_plagiarism["max_score_url"],
        "plagiarism_data": results_plagiarism["data"],
        "AI_category": result_AI["category"],
        "AI_flag": result_AI["AI"],
        "AI_percentage": result_AI["AI_Percentage"],
        "AI_confidence": result_AI["confidence"],
        "AI_confidence_difference": result_AI["confidence_difference"],
        "AI_probabilities": result_AI["probabilities"],
        "AI_metrics": result_AI["metrics"],
        "AI_threshold_used": result_AI["threshold_used"],
    }

    return result_data



import requests
from bs4 import BeautifulSoup
import hashlib
import json
import random
import nltk
from nltk.tokenize import sent_tokenize
from typing import List, Dict, Any

# Ensure NLTK resources are downloaded for sentence tokenization
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    print("Downloading 'punkt' package...")
    nltk.download('punkt_tab')

def get_useragent():
    _useragent_list = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.62',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/111.0'
    ]
    return random.choice(_useragent_list)

def google_search(query):
    links = []
    try:
        headers = {"User-Agent": get_useragent()}
        params = {"q": f'"{query}"', "num": 3, "hl": 'en'}

        # Perform the Google search request
        response = requests.get("https://www.google.com/search", headers=headers, params=params, timeout=5)
        response.raise_for_status()  # Raises an error for bad responses

        # Parse the response content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extract URLs from the search result blocks (avoiding ads, etc.)

        try:
            for result in soup.find_all('h3'):
                link = result.find_parent('a')
                if link:
                    link_text = link.get('href')
                    links.append(link_text)
        except:
            for link_tag in soup.select('div.yuRUbf a'):
                link = link_tag.get('href')
                if link:
                    links.append(link)


    except requests.RequestException as e:
        print(f"Error during the search request: {e}")

    return links

def fetch_web_content(url: str) -> str:
    """Fetch and clean the main content of a webpage."""
    try:
        headers = {"User-Agent": get_useragent()}
        response = requests.get(url, headers=headers, timeout=5)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        paragraphs = soup.find_all("p")
        return " ".join([para.get_text() for para in paragraphs])
    except requests.RequestException as e:
        print(f"Failed to fetch {url}: {e}")
        return ""

def get_shingles(text: str, k: int = 5) -> set:
    """Generate k-shingles (sets of k consecutive words) for a given text."""
    words = text.split()
    shingles = set()
    for i in range(len(words) - k + 1):
        shingle = " ".join(words[i:i + k])
        shingle_hash = hashlib.md5(shingle.encode("utf-8")).hexdigest()
        shingles.add(shingle_hash)
    return shingles

def similarity1(set1, set2):
    # Check if either set is empty to prevent division by zero
    if len(set1) == 0 or len(set2) == 0:
        return 0.0
    intersection = len(set1.intersection(set2))
    return (intersection / len(set1)) * 100

def calculate_jaccard_similarity(shingles1: set, shingles2: set) -> float:
    """Calculate Jaccard similarity between two sets of shingles."""
    intersection = shingles1.intersection(shingles2)
    union = shingles1.union(shingles2)

    # Avoid division by zero if both sets are empty
    if len(union) == 0:
        return 0.0

    return len(intersection) / len(union)

def check_sentence_plagiarism(sentence: str) -> Dict[str, Any]:
    """Check plagiarism for a single sentence by searching and comparing content."""
    result = {"sentence": sentence, "matches": []}
    urls = google_search(sentence)

    for url in urls:
        content = fetch_web_content(url)
        if content:
            original_shingles = get_shingles(sentence)
            content_shingles = get_shingles(content)
            # similarity = calculate_jaccard_similarity(original_shingles, content_shingles)
            similarity = similarity1(original_shingles, content_shingles)
            result["matches"].append({"url": url, "score": similarity})

    # Sort matches by score in descending order and take the highest
    if result["matches"]:
        result["matches"].sort(key=lambda x: x["score"], reverse=True)
        highest_match = result["matches"][0]
        result["highest_match"] = {"url": highest_match["url"], "score": highest_match["score"]}
    else:
        result["highest_match"] = {"url": None, "score": 0.0}

    return result

def plagiarism_checker(text: str) -> List[Dict[str, Any]]:
    """Run plagiarism check on each sentence in the input text."""
    sentences = sent_tokenize(text)
    results = [check_sentence_plagiarism(sentence) for sentence in sentences]
    return get_score(results)

def get_score(data):
    # Merging sentences by URL and score
    result = []
    total_score = 0
    max_score = 0
    max_score_url = ""
    total_sentences = len(data)

    for entry in data:
        url = entry['highest_match']['url']
        score = entry['highest_match']['score']
        sentence = entry['sentence']

        # Check if score is greater than 40 for plagiarism flag
        plagiarism = True if score > 40 else False

        # Add score to total score
        total_score += score

        # Track maximum score and corresponding URL
        if score > max_score:
            max_score = score
            max_score_url = url

        # Check if we should merge with the last entry
        if result and result[-1]['url'] == url and result[-1]['score'] == score:
            result[-1]['sentence'] += " " + sentence  # Append the sentence
        else:
            # Add a new entry with plagiarism flag
            result.append({
                'sentence': sentence,
                'url': url,
                'score': score,
                'plagiarism': plagiarism
            })

    # Calculate average score
    average_score = total_score / total_sentences if total_sentences else 0

    # Create the final structured response
    response = {
        "plagiarism": any(entry['plagiarism'] for entry in result),
        "average_score": average_score,
        "max_score": max_score,
        "max_score_url": max_score_url,
        "data": result
    }

    return response


# input_text = """
# An Operating System can be defined as an interface between user and hardware. It is responsible for the execution of all the processes, Resource Allocation, CPU ..."""
# results_plagiarism= plagiarism_checker(input_text)
# print(json.dumps(results_plagiarism, indent=2))




from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

def detect_ai_generated_text_advanced_v3(
    text,
    model_name="roberta-base-openai-detector",
    threshold=0.5,
    max_length=512
):
    """
    Advanced detection of AI-generated text with detailed metrics, AI flag, and percentage.

    Parameters:
        text (str): Input text to analyze.
        model_name (str): Pre-trained model to use for detection.
        threshold (float): Confidence threshold for categorization (default: 0.5).
        max_length (int): Maximum length for tokenization (default: 512).

    Returns:
        dict: Detailed output with categorization, confidence, metrics, AI flag, and percentage.
    """
    try:
        # Load tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name)

        # Tokenize and prepare text
        tokens = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
        token_count = len(tokens["input_ids"][0])

        # Inference
        outputs = model(**tokens)
        probabilities = torch.softmax(outputs.logits, dim=1).detach().numpy()[0]
        ai_confidence = probabilities[1]
        human_confidence = probabilities[0]
        category = "AI-generated" if ai_confidence > threshold else "Human-written"

        # AI flag and percentage
        is_ai = ai_confidence > threshold
        ai_percentage = round(ai_confidence * 100, 2)

        # Additional metrics
        char_count = len(text)
        word_count = len(text.split())
        avg_word_length = char_count / word_count if word_count > 0 else 0
        confidence_diff = abs(ai_confidence - human_confidence)

        return {
            "category": category,
            "AI": is_ai,
            "AI_Percentage": ai_percentage,
            "confidence": ai_confidence if is_ai else human_confidence,
            "confidence_difference": confidence_diff,
            "probabilities": {
                "Human-written": human_confidence,
                "AI-generated": ai_confidence
            },
            "metrics": {
                "character_count": char_count,
                "word_count": word_count,
                "average_word_length": round(avg_word_length, 2),
                "token_count": token_count,
                "max_token_length": max_length
            },
            "threshold_used": threshold
        }
    except Exception as e:
        return {"error": str(e)}

# Example usage
#text = "In a world driven by rapid technological advancement, artificial intelligence has emerged as a transformative force. From revolutionizing healthcare with predictive diagnostics to enhancing efficiency in industries through automation, AI continues to redefine the boundaries of what is possible. However, this unprecedented growth also poses ethical dilemmas, emphasizing the need for responsible innovation to ensure these technologies benefit humanity as a whole."
#result_AI = detect_ai_generated_text_advanced_v3(text, threshold=0.7)
#print(result)
#print()

def convert_numpy_types(obj):
    if isinstance(obj, np.generic):  # Check if it’s a numpy scalar
        return obj.item()  # Convert to native Python type
    return obj  # Leave other types unchanged

#
 load models for first time
load_models()  # Load models once

Loading models...
Models loaded successfully.


In [ ]:

import json
question = "What is the purpose of a file system?"
student_answer = "An operating system (OS) is system software that manages computer hardware and software resources, and provides common services for computer programs."
expected_answer = "A file system organizes and controls how data is stored and retrieved on a computer system."
marks = 5  # Full marks for the question

# Get the evaluation data
evaluation_result = GetData(question, student_answer, expected_answer, marks)
#print(evaluation_result)

#result_json = json.dumps(evaluation_result, indent=4)
#print(result_json)

result_json = json.dumps(evaluation_result, indent=4, default=convert_numpy_types)
print(result_json)

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{
    "question": "What is the purpose of a file system?",
    "student_answer": "An operating system (OS) is system software that manages computer hardware and software resources, and provides common services for computer programs.",
    "expected_answer": "A file system organizes and controls how data is stored and retrieved on a computer system.",
    "marks": 5,
    "score": 1.0,
    "score_percentage": 20.0,
    "feedback": "The provided answer does not directly address the question about the purposes of file systems within an OS. The response discusses the general role of an operating systems which is relevant but too broad.",
    "similarity": 0.4761287569999695,
    "similarity_percentage": 47.61,
    "final_marks_obtained": 1.8532126708417516,
    "final_marks_percentage": 37.06,
    "student_word_count": 21,
    "student_sentence_count": 1,
    "plagiarism": true,
    "average_score": 100.0,
    "max_score": 100.0,
    "max_score_url": "https://en.wikipedia.org/wiki/Operating

In [ ]:

import csv
import json

# Function to save the data to CSV
def save_to_csv(data, filename):
    fieldnames = ["question", "student_answer", "expected_answer", "marks", "score", "score_percentage",
                  "feedback", "similarity", "similarity_percentage", "final_marks_obtained",
                  "final_marks_percentage", "student_word_count", "student_sentence_count", "plagiarism",
                  "average_score", "max_score", "max_score_url", "AI_category", "AI_flag",
                  "AI_percentage", "AI_confidence", "AI_confidence_difference", "AI_probabilities",
                  "AI_metrics", "AI_threshold_used"]

    # Ensure the data contains only the keys in fieldnames
    # This step filters out extra fields like 'plagiarism_data' if present
    filtered_data = {key: data[key] for key in fieldnames if key in data}

    # Append the evaluation result to CSV
    with open(filename, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        # Check if the file is empty, if so, write header
        file.seek(0, 2)  # Move to end of file
        if file.tell() == 0:  # File is empty
            writer.writeheader()

        writer.writerow(filtered_data)


# Data containing questions, student answers, expected answers, and marks
# Data containing questions, student answers, expected answers, and marks
Data = [
    {"question": "What is the purpose of a file system?",
     "student_answer": "The purpose of a file system is to provide a way to store, organize, and manage data on storage devices such as hard drives or SSDs. It defines how data is stored in files and directories, and handles the processes of reading, writing, and organizing data in a way that allows users and programs to easily access it. However, not all file systems offer the same level of performance or features, and some may not be efficient for handling large files or directories.",
     "expected_answer": "A file system organizes and controls how data is stored and retrieved on a computer system.",
     "marks": 5},

    {"question": "What is a process in an operating system?",
     "student_answer": "A process is an instance of a program that is being executed. It consists of the program's code, its current activity, and its resources such as memory and CPU time. The operating system manages processes to ensure that each gets a fair share of the CPU and that resources are allocated effectively. Processes may also have states like running, waiting, or terminated, depending on what they are doing at any given time. Processes are fundamental for multitasking in an operating system.",
     "expected_answer": "A process is a program in execution, consisting of code, data, and system resources.",
     "marks": 5},

    {"question": "Explain virtual memory.",
     "student_answer": "Virtual memory is a memory management capability that allows an operating system to use hardware and software to provide an 'idealized' abstraction of the storage resources that are actually available on a given machine. By swapping data between physical memory (RAM) and disk storage, the operating system can run larger programs or multiple programs simultaneously, even if there isn’t enough physical RAM. Virtual memory can sometimes cause slower performance due to disk access speed limitations, but it allows programs to run as if they have more memory than physically available.",
     "expected_answer": "Virtual memory allows a computer to compensate for physical memory shortages by temporarily transferring data from RAM to disk storage.",
     "marks": 5},

    {"question": "What is a deadlock?",
     "student_answer": "Deadlock occurs when two or more processes are unable to continue executing because they are each waiting for resources that the other processes are holding. This creates a situation where no process can progress, as they are all waiting indefinitely. In many cases, deadlocks can cause the system to become unresponsive or inefficient. Operating systems use various techniques like deadlock prevention, detection, and recovery to handle deadlocks. However, it remains a significant challenge in concurrent systems.",
     "expected_answer": "A deadlock is a situation where two or more processes are blocked forever, waiting for each other to release resources.",
     "marks": 5},

    {"question": "What are the different types of scheduling algorithms?",
     "student_answer": "Scheduling algorithms are used by the operating system to manage the execution of processes on the CPU. There are several types, including First-Come-First-Serve (FCFS), which executes processes in the order they arrive; Round Robin (RR), which allocates a fixed time slice to each process before moving to the next; and Shortest Job First (SJF), which executes the process with the shortest total runtime first. Each algorithm has its strengths and weaknesses, such as FCFS being easy to implement but potentially inefficient in terms of turnaround time. In contrast, Round Robin is more balanced but may lead to more frequent context switches.",
     "expected_answer": "The main types of CPU scheduling algorithms include First-Come-First-Serve (FCFS), Shortest Job Next (SJN), and Round Robin.",
     "marks": 5},

    {"question": "Define a thread.",
     "student_answer": "A thread is the smallest unit of execution within a process. It shares the process's resources, such as memory space, but has its own execution path. Threads allow a program to perform multiple operations concurrently, improving the efficiency and responsiveness of the system. Multithreading can be used for tasks such as processing multiple inputs or handling different aspects of a program simultaneously. Threads are often used in parallel processing to perform tasks more efficiently than a single-threaded program, although managing them properly can be complex.",
     "expected_answer": "A thread is the smallest unit of execution within a process.",
     "marks": 5},

    {"question": "What is the difference between a process and a thread?",
     "student_answer": "A process is a self-contained unit of execution with its own memory space and resources, whereas a thread is a component of a process. While processes are independent, threads within a process share resources such as memory. Threads allow a process to perform multiple tasks concurrently, but processes cannot directly share resources with each other. Processes are typically more expensive in terms of system resources, while threads are lightweight and allow for better performance in multi-tasking environments.",
     "expected_answer": "A process is an independent program with its own resources, while a thread is a component of a process that shares resources.",
     "marks": 5},

    {"question": "Explain the concept of paging.",
     "student_answer": "Paging is a memory management technique that breaks physical memory into fixed-sized blocks called pages. When a process needs to access data, the operating system maps logical addresses to physical addresses in memory. If the required page is not in memory, the operating system loads it from disk storage into an available page frame in memory. This process allows programs to run even if there isn't enough physical memory available, although it can lead to performance issues due to page faults and disk access time.",
     "expected_answer": "Paging is a memory management scheme that eliminates the need for contiguous allocation of physical memory.",
     "marks": 5},

    {"question": "What is the role of the operating system in managing hardware?",
     "student_answer": "The operating system plays a crucial role in managing the hardware of a computer by acting as an intermediary between hardware devices and the software applications running on the system. It ensures that hardware resources like the CPU, memory, and storage are allocated efficiently and fairly among all running processes. The operating system also provides device drivers to communicate with peripheral hardware like printers, keyboards, and monitors. Without an operating system, it would be difficult for users and applications to interact with hardware directly.",
     "expected_answer": "The operating system acts as an intermediary between hardware and software, managing resources like CPU, memory, and storage.",
     "marks": 5},

    {"question": "What is file fragmentation?",
     "student_answer": "File fragmentation occurs when files are stored in non-contiguous blocks on a disk, which can happen over time as files are modified, deleted, or resized. Fragmentation leads to reduced performance, as the system must search through scattered locations to access a file's contents. Defragmentation tools are often used to reorganize the data and reduce fragmentation, improving system performance. However, fragmentation is less of an issue with modern file systems and solid-state drives (SSDs), which do not have mechanical parts.",
     "expected_answer": "File fragmentation occurs when a file is split into pieces and scattered across the storage device.",
     "marks": 5},

    {"question": "What is a system call?",
     "student_answer": "A system call is a mechanism that allows programs to request services from the operating system's kernel. For example, when a program wants to read or write data to a file, it makes a system call to request that the kernel perform the operation on its behalf. System calls provide a controlled interface for interacting with the underlying hardware and system resources, ensuring that programs don't directly manipulate sensitive parts of the operating system.",
     "expected_answer": "A system call is a programmatic way a program requests a service from the operating system's kernel.",
     "marks": 5},

    {"question": "Explain the concept of inter-process communication.",
     "student_answer": "Inter-process communication (IPC) refers to the methods used by processes to exchange data and synchronize their activities. It is important for multi-process systems, where processes often need to share information or coordinate actions. IPC can be achieved using mechanisms like shared memory, message queues, semaphores, or sockets. While IPC is powerful, it requires careful management to avoid issues like deadlock or race conditions, which can lead to unpredictable behavior or system crashes.",
     "expected_answer": "Inter-process communication (IPC) is the mechanism that allows processes to communicate and synchronize their actions.",
     "marks": 5},

    {"question": "What is the significance of a kernel in an operating system?",
     "student_answer": "The kernel is the core part of the operating system that manages system resources such as CPU, memory, and input/output devices. It acts as a bridge between applications and hardware, ensuring that processes run smoothly and that resources are allocated efficiently. The kernel is responsible for low-level tasks like memory management, process scheduling, and interrupt handling. A well-designed kernel can significantly improve the overall performance and stability of an operating system.",
     "expected_answer": "The kernel is the core part of the OS that manages system resources and provides essential services for programs.",
     "marks": 5},

    {"question": "What is memory management?",
     "student_answer": "Memory management is the process of managing the computer's memory resources, ensuring that each program and process gets the memory it needs to function. This includes tasks such as allocating memory for processes, tracking memory usage, and swapping data between RAM and disk storage when necessary. Effective memory management is critical for system performance, as poor memory handling can lead to issues like fragmentation or memory leaks, which can slow down or crash the system.",
     "expected_answer": "Memory management involves the process of controlling and coordinating computer memory, including the allocation and deallocation of memory blocks.",
     "marks": 5},

    {"question": "What are device drivers?",
     "student_answer": "Device drivers are specialized programs that enable the operating system to communicate with and control hardware devices. They act as a translator between the hardware and the operating system, converting general OS commands into device-specific instructions. For example, a printer driver translates print commands into a format the printer understands. Without device drivers, the operating system would not be able to interact with hardware peripherals, making them essential for system functionality.",
     "expected_answer": "Device drivers are software components that allow the operating system to communicate with hardware peripherals.",
     "marks": 5}
]

# File path where you want to save the CSV
filename = '/content/drive/MyDrive/operating_system_exam_results.csv'  # Modify path if necessary

# Loop through the Data and save each result
for record in Data:
    question = record["question"]
    student_answer = record["student_answer"]
    expected_answer = record["expected_answer"]
    marks = record["marks"]

    # Get the evaluation result
    evaluation_result = GetData(question, student_answer, expected_answer, marks)

    # Save to CSV
    save_to_csv(evaluation_result, filename)

    # Print the result for the current record
    print(f"Saved evaluation result for question: {question}")
    #print(evaluation_result)

    result_json = json.dumps(evaluation_result, indent=4, default=convert_numpy_types)
    print(result_json)
    print("------")

print("Data has been saved to CSV.")

Failed to fetch https://quizlet.com/ca/287789983/chapter-34-memory-management-file-systems-flash-cards/: 403 Client Error: Forbidden for url: https://quizlet.com/ca/287789983/chapter-34-memory-management-file-systems-flash-cards/


Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Saved evaluation result for question: What is the purpose of a file system?
{
    "question": "What is the purpose of a file system?",
    "student_answer": "The purpose of a file system is to provide a way to store, organize, and manage data on storage devices such as hard drives or SSDs. It defines how data is stored in files and directories, and handles the processes of reading, writing, and organizing data in a way that allows users and programs to easily access it. However, not all file systems offer the same level of performance or features, and some may not be efficient for handling large files or directories.",
    "expected_answer": "A file system organizes and controls how data is stored and retrieved on a computer system.",
    "marks": 5,
    "score": 3.0,
    "score_percentage": 60.0,
    "feedback": "The answer provided a clear explanation of the primary function of file management systems without including extraneous details about other aspects of computer science like n

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Saved evaluation result for question: What is a process in an operating system?
{
    "question": "What is a process in an operating system?",
    "student_answer": "A process is an instance of a program that is being executed. It consists of the program's code, its current activity, and its resources such as memory and CPU time. The operating system manages processes to ensure that each gets a fair share of the CPU and that resources are allocated effectively. Processes may also have states like running, waiting, or terminated, depending on what they are doing at any given time. Processes are fundamental for multitasking in an operating system.",
    "expected_answer": "A process is a program in execution, consisting of code, data, and system resources.",
    "marks": 5,
    "score": 2.0,
    "score_percentage": 40.0,
    "feedback": "The answer provides a good explanation of processes but does not fully capture the key aspects of how an OS manages them.",
    "similarity": 0.88120293

...

Saved evaluation result for question: What is the significance of a kernel in an operating system?
{
    "question": "What is the significance of a kernel in an operating system?",
    "student_answer": "The kernel is the core part of the operating system that manages system resources such as CPU, memory, and input/output devices. It acts as a bridge between applications and hardware, ensuring that processes run smoothly and that resources are allocated efficiently. The kernel is responsible for low-level tasks like memory management, process scheduling, and interrupt handling. A well-designed kernel can significantly improve the overall performance and stability of an operating system.",
    "expected_answer": "The kernel is the core part of the OS that manages system resources and provides essential services for programs.",
    "marks": 5,
    "score": 2.0,
    "score_percentage": 40.0,
    "feedback": "The explanation provides a good overview of what a Linux kernel does but lacks de

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Saved evaluation result for question: What is memory management?
{
    "question": "What is memory management?",
    "student_answer": "Memory management is the process of managing the computer's memory resources, ensuring that each program and process gets the memory it needs to function. This includes tasks such as allocating memory for processes, tracking memory usage, and swapping data between RAM and disk storage when necessary. Effective memory management is critical for system performance, as poor memory handling can lead to issues like fragmentation or memory leaks, which can slow down or crash the system.",
    "expected_answer": "Memory management involves the process of controlling and coordinating computer memory, including the allocation and deallocation of memory blocks.",
    "marks": 5,
    "score": 2.0,
    "score_percentage": 40.0,
    "feedback": "The answer provides a basic understanding of memory allocation but does not cover all aspects of modern memory-management

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Saved evaluation result for question: What are device drivers?
{
    "question": "What are device drivers?",
    "student_answer": "Device drivers are specialized programs that enable the operating system to communicate with and control hardware devices. They act as a translator between the hardware and the operating system, converting general OS commands into device-specific instructions. For example, a printer driver translates print commands into a format the printer understands. Without device drivers, the operating system would not be able to interact with hardware peripherals, making them essential for system functionality.",
    "expected_answer": "Device drivers are software components that allow the operating system to communicate with hardware peripherals.",
    "marks": 5,
    "score": 3.0,
    "score_percentage": 60.0,
    "feedback": null,
    "similarity": 0.9291216135025024,
    "similarity_percentage": 92.91,
    "final_marks_obtained": 3.543359137899424,
    "final_mar